In [1]:
import os
import torch
import random
from PIL import Image
import my_prompt5_MA as my_prompt
from file_managing import (
    load_selected_samples,
    get_actual_path,
    get_gt_path,
)
from config import AGD20K_PATH, model_name
from VLM_model_dot import QwenVLModel, MetricsTracker
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["PYTORCH_ENABLE_SDPA"] = "1"


/home/bongo/anaconda3/envs/qwen25/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
missing_gt = 0
model = QwenVLModel(model_name = model_name)
metrics_tracker_ego = MetricsTracker(name="only_ego")
metrics_tracker_exo_best = MetricsTracker(name="with_exo_best")

json_path = os.path.join("selected_samples.json")
data = load_selected_samples(json_path)


💻 사용 디바이스: cuda
🤖 Qwen/Qwen2.5-VL-3B-Instruct 모델 로딩중...


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.33it/s]


✅ 모델 로딩 완료!


In [4]:
# Get total number of samples
total_samples = len(data['selected_samples'])

# Process each sample
print(f"Processing {total_samples} samples...")
print("=" * 50)    
for pair_key, sample_info in data["selected_samples"].items():
    print("--- Start  ", "-"*80) 
    
    action = sample_info["action"]
    object_name = sample_info["object"]
    break
print(f"{action}, {object_name}")

Processing 123 samples...
--- Start   --------------------------------------------------------------------------------
jump, skis


In [5]:
image_path = get_actual_path(sample_info["image_path"])
gt_path = get_gt_path(image_path)    
print(f"Action : {action}, Object : {object_name} image_name : {image_path.split('/')[-1]}")
exo_best_path = "dogs.jpg"

Action : jump, Object : skis image_name : skis_002829.jpg


In [6]:
prompt1 = my_prompt.step1(action, object_name)
prompt1


'\n    # MISSION: GENERATOR AGENT\n\nYou are the **Generator Agent**. Your task is to analyze the provided images (egocentric) and the task description. Generate a **dense cloud of all plausible keypoints** in the **egocentric image** where the action could occur. Be exhaustive and do not filter or judge the points yet.\n\n---\n\n## INPUTS\n\n* **Egocentric Image:** [Primary image for annotation]\n* **Task:** Perform the action \'jump\' on the \'skis\'.\n\n---\n\n## OUTPUT FORMAT (Strict JSON)\n\nProvide your answer inside a single JSON block with the key "generated_points".\n\n```json\n{\n  "generated_points": [\n    [x1, y1],\n    [x2, y2],\n    [x3, y3]\n  ]\n}```\n\n'

In [7]:
results = model.process_image_ego(image_path, prompt1, gt_path, action)
results

qwen ego Results!! : ```json
{
  "generated_points": [
    [105, 1468],
    [190, 1468],
    [275, 1468],
    [360, 1468],
    [445, 1468]
  ]
}
```
parsed dots!!! : [[105, 1468], [190, 1468], [275, 1468], [360, 1468], [445, 1468]]


{'text_result': '```json\n{\n  "generated_points": [\n    [105, 1468],\n    [190, 1468],\n    [275, 1468],\n    [360, 1468],\n    [445, 1468]\n  ]\n}\n```',
 'dots': [[105, 1468], [190, 1468], [275, 1468], [360, 1468], [445, 1468]],
 'dot_image_path': '/home/bongo/porter_notebook/research/new_qwen_AG/dot_images/only_ego/skis_002829_jump.jpg',
 'dot_only_image_path': '/home/bongo/porter_notebook/research/new_qwen_AG/dot_images/dots_only/skis_002829_jump_dots.jpg',
 'heatmap_image_path': '/home/bongo/porter_notebook/research/new_qwen_AG/dot_images/heatmaps/skis_002829_jump_heatmap.jpg',
 'heatmap_tensor': tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.2444, 0.2478, 0.2512,  ..., 0.2185, 0.2153, 0.2122],
         [0.2437, 0.2470, 0.2504,  ..., 0.2178, 0.2146, 0.2115],
         [0.2429, 0.2462, 0.2496,  ..., 0.2170, 0.2

In [8]:
prompt2 = my_prompt.step2(action, object_name)
prompt2



'\n# MISSION: INTERACTION PLANNER AGENT\n\nYou are the **Interaction Planner Agent**. Your task is to analyze the user\'s intended **Task** to determine the optimal **Affordance Topology**. The topology describes the geometric \'shape\' of the interaction.\n\n---\n\n## INPUTS\n\n* **Task:** Perform the action \'jump\' on the \'skis\'.\n\n---\n\n## INSTRUCTIONS\n\n1.  **Reasoning:** First, briefly explain your reasoning. What is the physical goal of the action \'jump\'? Does it require precise force (`POINT`), interaction along an edge (`LINE/CURVE`), or convenience across an area (`REGION`)?\n2.  **Decision:** Then, choose one of the three topology types.\n\n---\n\n## OUTPUT FORMAT (Strict JSON)\n\nProvide your answer inside a single JSON block with the key "topology_analysis".\n\n```json\n{\n  "topology_analysis": {\n    "rationale": "The action \'open refrigerator\' prioritizes convenience, allowing the user to pull from anywhere on the handle. Therefore, the interaction space is a f

In [9]:
results2 = model.ask(prompt2)
results2

'```json\n{\n  "topology_analysis": {\n    "rationale": "The action \'jump\' on skis typically involves a combination of force and movement over a specific area. This requires precision in force application and movement along a path, which aligns more closely with the \'POINT\' topology where the action is performed at a specific location.",\n    "topology": "POINT"\n  }\n}\n```'

In [10]:
prompt3 = my_prompt.step3(action, object_name)
prompt3

'\n# MISSION: REFINER AGENT\n\nYou are the **Refiner Agent**. Your task is to take the dense point cloud from the Generator and the strategic topology decision from the Planner to produce the final, precise set of keypoints.\n\n---\n\n## INPUTS\n\n* **Generated Points:** A list of all plausible keypoints, e.g., `[[x1, y1], [x2, y2], ...]`\n* **Topology Analysis:** The decision from the planner, e.g., `{ "topology": "REGION", "rationale": "..." }`\n\n---\n\n## INSTRUCTIONS\n\nApply the topology decision to filter the generated points using the following logic:\n\n* **If `topology` is `POINT`:** Select the **single most optimal point** from the candidates that best achieves the action\'s goal.\n* **If `topology` is `LINE/CURVE`:** Select the subset of points that form the most relevant **continuous line or curve**. Discard outliers.\n* **If `topology` is `REGION`:** Select **all points** that fall within the primary functional area. Discard any stray points outside this main cluster.\n\n

In [6]:

results = model.process_image_exo(image_path, prompt, gt_path, exo_best_path, action)
results

exo file name : dogs.jpg / exo_path
final points :[[105, 1468], [110, 1470], [115, 1472], [120, 1474], [125, 1476], [130, 1478], [135, 1480], [140, 1482], [145, 1484], [150, 1486], [155, 1488], [160, 1490], [165, 1492], [170, 1494], [175, 1496], [180, 1498], [185, 1500], [190, 1502], [195, 1504], [200, 1506], [205, 1508], [210, 1510], [215, 1512], [220, 1514], [225, 1516], [230, 1518], [235, 1520], [240, 1522], [245, 1524], [250, 1526], [255, 1528], [260, 1530], [265, 1532], [270, 1534], [275, 1536], [280, 1538], [285, 1540], [290, 1542], [295, 1544], [300, 1546], [305, 1548], [310, 1550], [315, 1552], [320, 1554], [325, 1556], [330, 1558], [335, 1560], [340, 1562], [345, 1564], [350, 1566], [355, 1568], [360, 1570], [365, 1572], [370, 1574], [375, 1576], [380, 1578], [385, 1580], [390, 1582], [395, 1584], [400, 1586], [405, 1588], [410, 1590], [415, 1592], [420, 1594], [425, 1596], [430, 1598], [435, 1599], [440, 1601], [445, 1603], [450, 1605], [455, 1607], [460, 1609], [465, 1611], 

{'text_result': '```json\n{\n  "generated_points": [\n    [105, 1468],\n    [110, 1470],\n    [115, 1472],\n    [120, 1474],\n    [125, 1476],\n    [130, 1478],\n    [135, 1480],\n    [140, 1482],\n    [145, 1484],\n    [150, 1486],\n    [155, 1488],\n    [160, 1490],\n    [165, 1492],\n    [170, 1494],\n    [175, 1496],\n    [180, 1498],\n    [185, 1500],\n    [190, 1502],\n    [195, 1504],\n    [200, 1506],\n    [205, 1508],\n    [210, 1510],\n    [215, 1512],\n    [220, 1514],\n    [225, 1516],\n    [230, 1518],\n    [235, 1520],\n    [240, 1522],\n    [245, 1524],\n    [250, 1526],\n    [255, 1528],\n    [260, 1530],\n    [265, 1532],\n    [270, 1534],\n    [275, 1536],\n    [280, 1538],\n    [285, 1540],\n    [290, 1542],\n    [295, 1544],\n    [300, 1546],\n    [305, 1548],\n    [310, 1550],\n    [315, 1552],\n    [320, 1554],\n    [325, 1556],\n    [330, 1558],\n    [335, 1560],\n    [340, 1562],\n    [345, 1564],\n    [350, 1566],\n    [355, 1568],\n    [360, 1570],\n    [365, 